# Convolutional Neural Network

### Importing the libraries

In [2]:
!pip3 install --upgrade tensorflow keras Pillow

Using legacy 'setup.py install' for clang, since package 'wheel' is not installed.
Using legacy 'setup.py install' for termcolor, since package 'wheel' is not installed.
Using legacy 'setup.py install' for wrapt, since package 'wheel' is not installed.
    Running setup.py install for clang: started
    Running setup.py install for clang: finished with status 'done'
    Running setup.py install for termcolor: started
    Running setup.py install for termcolor: finished with status 'done'
    Running setup.py install for wrapt: started
    Running setup.py install for wrapt: finished with status 'done'


You should consider upgrading via the 'c:\users\lucas\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [40]:
# https://fairyonice.github.io/Learn-about-ImageDataGenerator.html
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,         # Rescala imagem
                                   shear_range = 0.2,        # Aplica um efeito de rotação
                                   zoom_range = 0.2,         # Aplica um zoom a imagem, com intensidades variadas
                                   horizontal_flip = True,   # Espelha imagem
                                   # Adicionados
                                   #rotation_range=20,        # Rotacao aleatoria de X graus
                                   width_shift_range=0.2)   # Move a imagem para os lados

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 64,  # Alterado de 32 para 64
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [41]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 6,  # Alterado de 32 para 64 
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [42]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [43]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [44]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [45]:
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [46]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [47]:
cnn.add(tf.keras.layers.Dense(units=256, activation='relu')) # Alterado units de 126 para 256

### Step 5 - Output Layer

In [48]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [49]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [50]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 30) # Alterado de 25 para 50

Epoch 1/30
125/125 [==============================] - 27s 215ms/step - loss: 0.6713 - accuracy: 0.5681 - val_loss: 0.6892 - val_accuracy: 0.5535
Epoch 2/30
125/125 [==============================] - 26s 210ms/step - loss: 0.6037 - accuracy: 0.6758 - val_loss: 0.5649 - val_accuracy: 0.7200
Epoch 3/30
125/125 [==============================] - 26s 209ms/step - loss: 0.5741 - accuracy: 0.6952 - val_loss: 0.6043 - val_accuracy: 0.6810
Epoch 4/30
125/125 [==============================] - 27s 214ms/step - loss: 0.5505 - accuracy: 0.7188 - val_loss: 0.5197 - val_accuracy: 0.7490
Epoch 5/30
125/125 [==============================] - 27s 214ms/step - loss: 0.5337 - accuracy: 0.7308 - val_loss: 0.5264 - val_accuracy: 0.7320
Epoch 6/30
125/125 [==============================] - 26s 208ms/step - loss: 0.5191 - accuracy: 0.7409 - val_loss: 0.5829 - val_accuracy: 0.7000
Epoch 7/30
125/125 [==============================] - 26s 209ms/step - loss: 0.5055 - accuracy: 0.7541 - val_loss: 0.4688 - val_ac

## Part 4 - Making a single prediction

In [53]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [54]:
print(prediction)

cat


## Fim